In [9]:
# Importing Packages
import os
import tarfile
import pandas as pd
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [10]:
# Base Folder of all Twitter tar files
twitter_folder = 'E:/Study Material/Data Mining and Machine Learning - 2/Project/Data'
twitter_data = os.listdir(twitter_folder)
print(twitter_data)

['August', 'Final', 'twitter_stream_2019_08_01.tar', 'twitter_stream_2019_08_02.tar', 'twitter_stream_2019_08_03.tar', 'twitter_stream_2019_08_04.tar', 'twitter_stream_2019_08_05.tar', 'twitter_stream_2019_08_06.tar', 'twitter_stream_2019_08_07.tar', 'twitter_stream_2019_08_08.tar', 'twitter_stream_2019_08_09.tar', 'twitter_stream_2019_08_10.tar', 'twitter_stream_2019_08_11.tar', 'twitter_stream_2019_08_12.tar', 'twitter_stream_2019_08_13.tar', 'twitter_stream_2019_08_14.tar', 'twitter_stream_2019_08_15.tar', 'twitter_stream_2019_08_16.tar', 'twitter_stream_2019_08_17.tar', 'twitter_stream_2019_08_18.tar', 'twitter_stream_2019_08_19.tar', 'twitter_stream_2019_08_20.tar', 'twitter_stream_2019_08_21.tar', 'twitter_stream_2019_08_22.tar', 'twitter_stream_2019_08_23.tar', 'twitter_stream_2019_08_24.tar', 'twitter_stream_2019_08_25.tar', 'twitter_stream_2019_08_26.tar', 'twitter_stream_2019_08_27.tar', 'twitter_stream_2019_08_28.tar', 'twitter_stream_2019_08_29.tar', 'twitter_stream_2019_08

In [ ]:
# Extraction of tarfiles
for i in range(0, len(twitter_data)):
    if tarfile.is_tarfile(twitter_folder+"/"+twitter_data[i]):
        tf = tarfile.open(twitter_folder+"/"+twitter_data[i])
        tf.extractall(path = twitter_folder+"/August")
        tf.close()

In [36]:
# Creating the environment path for all json.bz2 files
days = os.listdir(twitter_folder+"/August/")
abspathev = []
for i in range(0,1):
    hour = os.listdir(twitter_folder+"/August/"+days[i])
    for j in range(0,len(hour)):
        minute = os.listdir(twitter_folder+"/August/"+days[i]+"/"+hour[j])
        for k in range(0,len(minute)):
            abspathev.append(os.path.abspath(twitter_folder+"/August/"+days[i]+"/"+hour[j]+"/"+minute[k]))
len(abspathev)

1411

In [18]:
# Enabling Spark Context and creating Spark Session
#sc = SparkContext()
spark = SparkSession.Builder().getOrCreate()

In [30]:
# Reading all json.bz2 files and creating a spark dataframe
df = spark.read.json(abspathev)

In [32]:
# Creating a sql view
df.createOrReplaceTempView("tweet")

In [33]:
# Extracting only required data from the spark dataframe
sqlDF = spark.sql("SELECT created_at, array_join(entities.hashtags.text, ',') as hashtags, id as tweetid, lang, text as tweet, user.id as userid FROM tweet where lang like ('%en%') and text not like ('%RT%')")

In [34]:
# Filtering Tweets that have non-empty hashtags (August - 33001619)
sqlDF = sqlDF.filter("hashtags != ''")

41292


In [38]:
# Writing data to csv file
twitter_df = sqlDF.toPandas()
twitter_df.to_csv(r'E:\Study Material\Data Mining and Machine Learning - 2\Project\Data\Final\August01_Tweets_Final.csv', index=False, header=True)